In [ ]:
# This is a simple python code that uses Pinecone and also uses Lanhchain to develop and maintain chat history. 

In [ ]:
import os
import uuid
from dotenv import load_dotenv
from loguru import logger

from langchain_google_genai import ChatGoogleGenerativeAI
from railguard import RailGuard
from pinecone_code import Pinecone_code

# Langgraph and message classes
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, HumanMessage

#https://python.langchain.com/docs/how_to/chatbots_memory/

[nltk_data] Downloading package punkt to /Users/jenitza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class LanggraphAssistant:
    def __init__(self):
       
        self.logger = logger
        self.logger.add("logs/capstone.log", format="{time} {level} {message}", level="TRACE")
        load_dotenv()

        # Initialize RailGuard for safety or evaluation
        self.railguard = RailGuard()

        # Initialize the Gemini model via ChatGoogleGenerativeAI
        api_key = os.getenv("GOOGLE_API_KEY")
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",
            temperature=0.3,
            google_api_key=api_key
        )

        # Build the langgraph workflow
        self.workflow = StateGraph(state_schema=MessagesState)

        # Function that calls the model with current messages
        def call_model(state: MessagesState):
            response = self.llm.invoke(state["messages"])
            return {"messages": response}

        # Define the graph edge and node
        self.workflow.add_edge(START, "model")
        self.workflow.add_node("model", call_model)

        # Initialize memory to store conversation history.
        self.memory = MemorySaver()
        self.app = self.workflow.compile(checkpointer=self.memory)

        # Initialize your Pinecone interface
        self.pinecone = Pinecone_code()

        # Create a persistent thread id to preserve the conversation state
        self.thread_id = uuid.uuid4()
        self.config = {"configurable": {"thread_id": self.thread_id}}

        # Track the number of questions to determine if context should be added.
        self.question_count = 0

    def clean_pinecone_response(self, pinecone_response):
        return [match['metadata']['text'] for match in pinecone_response.get('matches', [])]

    def ask_question(self, question: str) -> str:
        # Evaluate the question with RailGuard first
        railguard_response = self.railguard.railguard_eval(question)
        
        # Comment the not part if you want to test for follow up questions. Currently as the railguard prompt does not allow for follou up questions, this would not pass. Else modify rail guard prompt.
        if not railguard_response:
            return "Please ask a valid question related to law."

        self.question_count += 1
        try:
            # For the very first question, enrich with context from Pinecone
            if self.question_count == 1:
                pinecone_response = self.pinecone.query_pinecone(question, top_k=10) # Replace your RAG HERE
                context = self.clean_pinecone_response(pinecone_response)
                context_str = "\n".join(context)
                system_message = SystemMessage(
                    content=f"You are a legal assistant. Use this context to answer:\n{context_str}"
                )
                messages = [system_message, HumanMessage(content=question)]
            else:
                # For subsequent questions, just provide the human message.
                messages = [HumanMessage(content=question)]
            
            result_messages = []
            # Stream the conversation. The MemorySaver ensures that previous messages (and responses)
            # are recalled using the same thread_id in the config.
            for event in self.app.stream({"messages": messages}, self.config, stream_mode="values"):
                result_messages = event["messages"]
        
           
            return result_messages[-1].content
        except Exception as e:
            self.logger.error(f"Chat error: {e}")
            return "Error processing your request"

In [4]:
if __name__ == "__main__":
    assistant = LanggraphAssistant()

    # First question 
    answer1 = assistant.ask_question("What is Optional Practical Training (OPT)?")
    print("Answer to first question:", answer1)

    # Second question asking about the first question to see history preserved.
    answer2 = assistant.ask_question("What was my first question?")
    print("Answer to second question:", answer2)

Answer to first question: Optional Practical Training (OPT) is a type of work permission available for eligible F-1 students. It allows students to gain real-world work experience that is directly related to their field of study. It can be either pre-completion (before the program end date, part-time or full-time) or post-completion (after the program end date, at least 20 hours per week or full-time). A student is typically eligible for 12 months of OPT per higher level of study.
Answer to second question: Please ask a valid question related to law.
